+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [1]:
#Importar librerias
import pandas as pd
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#Traigo la tabla user_items para unirlas
df_user_items = pd.read_parquet('dfitemsAPI.parquet', engine='pyarrow')
df_user_items.head(2)


,item_id,item_name,playtime_forever,user_id,steam_id,items_count
2654179,242680,Nuclear Throne,0,76561198044831412,76561198044831412,281
3559660,252950,Rocket League,1875,redrider623,76561198070855384,197


In [3]:
df_user_items['user_id']=df_user_items['user_id'].astype('string') #paso de object a string

In [4]:
play_time_user= df_user_items[["item_id","playtime_forever","user_id"]] # me quedo con las variables necesarias
play_time_user.head(3)

,item_id,playtime_forever,user_id
2654179,242680,0,76561198044831412
3559660,252950,1875,redrider623
3278341,215530,1415,Remo191


In [5]:
#Traigo la tabla steam games la cual voy a cruzar con la de arriba.con el item_id
df_games = pd.read_parquet('dfgamesrecomendacionAPI.parquet', engine='pyarrow')
df_games.head(2)

,item_id,app_name,title,genres,release_date,developer,price
24777,384550,Painters Guild,Painters Guild,Management,2015,Lucas Molina,9.99
30485,420440,Atlantic Fleet,Atlantic Fleet,World War II,2016,Killerfish Games,9.99


In [6]:
# pongo la columnas que voy a usar.
tabladgenre = df_games[["item_id","genres","release_date"]]

In [7]:
#unifico
genre_user= play_time_user.merge(tabladgenre, on="item_id")
genre_user.head(5)

,item_id,playtime_forever,user_id,genres,release_date
0,242680,0,76561198044831412,Action,2015
1,242680,0,76561198044831412,Twin Stick Shooter,2015
2,242680,0,76561198044831412,Great Soundtrack,2015
3,252950,1875,redrider623,Team-Based,2015
4,252950,1875,redrider623,Sports,2015


In [8]:
genre_user['genres']=genre_user['genres'].astype('string') #pasar de object a string
genre_user['release_date']=genre_user['release_date'].astype('string')
genre_user['user_id']=genre_user['user_id'].astype('string')

In [9]:
#agrupo el genero por usuario y horas jugadas
user_hours_year= genre_user.groupby(["genres","user_id","release_date"])["playtime_forever"].sum().reset_index()
user_hours_year

,genres,user_id,release_date,playtime_forever
0,1980s,2Ta4,2014,0
1,1980s,76561197966936422,2015,39
2,1980s,76561197967318124,2015,220
3,1980s,76561197967469174,2014,0
4,1980s,76561197978426585,2015,38
...,...,...,...,...
704595,e-sports,tobscene,1999,0
704596,e-sports,twinturboskyline,2004,200
704597,e-sports,ultimax10,2004,0
704598,e-sports,valdosta,2004,0


In [10]:
#necesito saber los usuarios con mayor horas jugadas por genero
user_hours_total= genre_user.groupby(["genres","user_id"])["playtime_forever"].sum().reset_index()
user_hours_total.head(3)

,genres,user_id,playtime_forever
0,1980s,2Ta4,0
1,1980s,76561197966936422,39
2,1980s,76561197967318124,220


In [11]:
#por cada genero el total de horas sumadas
max_user = user_hours_total.groupby(["genres"])["playtime_forever"].agg(playtime_forever="max").reset_index()
max_user

,genres,playtime_forever
0,1980s,6015
1,1990's,4415
2,2.5D,5539
3,2D,95889
4,2D Fighter,18920
...,...,...
293,Western,1315
294,Word Game,314
295,World War II,14216
296,Zombies,301383


In [12]:
tabla_user =pd.merge(max_user,user_hours_total)
tabla_user #ya tengo el usuario con mayor horas jugadas por genero, ahora me falta incluir el total por año

,genres,playtime_forever,user_id
0,1980s,6015,Milkshakes-
1,1990's,4415,926457123864182681994
2,2.5D,5539,Cow666
3,2D,95889,Brain_muncher
4,2D Fighter,18920,76561198047763701
...,...,...,...
293,Western,1315,76561198073082088
294,Word Game,314,sesepe
295,World War II,14216,taronzzz
296,Zombies,301383,ILDHAM123


In [13]:
user_hours_year2=user_hours_year.rename(columns={'playtime_forever':'playtime_forever_year'})
user_hours_year2.head(5)

,genres,user_id,release_date,playtime_forever_year
0,1980s,2Ta4,2014,0
1,1980s,76561197966936422,2015,39
2,1980s,76561197967318124,2015,220
3,1980s,76561197967469174,2014,0
4,1980s,76561197978426585,2015,38


In [14]:
#voy a probar si el cruce funciona ok, en el caso del usuario REBAS_AS_F-T	
prueba3=user_hours_year2[(user_hours_year2['user_id'] == 'REBAS_AS_F-T') & (user_hours_year2['genres']=='Indie')]
prueba3
#Para el usuario REBAS_AS_F-T la tabla tabla_user me debe traer 16 registros

,genres,user_id,release_date,playtime_forever_year
343330,Indie,REBAS_AS_F-T,2012,3126
343331,Indie,REBAS_AS_F-T,2013,0
343332,Indie,REBAS_AS_F-T,2014,2076
343333,Indie,REBAS_AS_F-T,2015,4076
343334,Indie,REBAS_AS_F-T,2016,7943
343335,Indie,REBAS_AS_F-T,2017,24


In [15]:
tabla_user2 =pd.merge(tabla_user,user_hours_year2)
tabla_user2

,genres,playtime_forever,user_id,release_date,playtime_forever_year
0,1980s,6015,Milkshakes-,2015,6015
1,1990's,4415,926457123864182681994,1997,4415
2,2.5D,5539,Cow666,1994,5539
3,2D,95889,Brain_muncher,2014,95889
4,2D Fighter,18920,76561198047763701,2013,18920
...,...,...,...,...,...
316,Western,1315,76561198073082088,2016,1315
317,Word Game,314,sesepe,2016,314
318,World War II,14216,taronzzz,2003,14216
319,Zombies,301383,ILDHAM123,2009,301383


In [16]:
#verificamos
prueba4=tabla_user2[(tabla_user2['user_id'] == 'REBAS_AS_F-T') & (tabla_user2['genres']=='Indie')]
prueba4.shape # esta ok
#ya tenemos la tabla para crear la funcion

(0, 5)

In [17]:
tabla_user2 = tabla_user2.rename(columns={'ref':'release_date', 'playtime_forever_year':'Horas jugadas'})
tabla_user2["release_date"] = pd.to_numeric(tabla_user2["release_date"])

In [20]:
# Guardar el DataFrame como un archivo Parquet
ruta_salida = 'tabla_user2.parquet'
tabla_user2.to_parquet(ruta_salida, engine='pyarrow')

In [18]:
def UserForGenre(genero):
    usuario= tabla_user2[tabla_user2["genres"]== genero]["user_id"].iloc[0] #obtengo usuario
    historial=tabla_user2[(tabla_user2['user_id'] == usuario) & (tabla_user2['genres']==genero)] #filtro por el genero y usuario
    historial2 = historial[['release_date', 'Horas jugadas']].copy() #me quedo con las columnas necesarias
    historial3=historial2.to_dict(orient="records")
    return {"Usuario":usuario ,"con más horas jugadas para": genero, "Historial acumulado": historial3 }

In [23]:
UserForGenre('Action') # si funciona la funcion 

{'Usuario': 'julius13',
 'con más horas jugadas para': 'Action',
 'Historial acumulado': [{'release_date': 2012, 'Horas jugadas': 0},
  {'release_date': 2013, 'Horas jugadas': 237539},
  {'release_date': 2015, 'Horas jugadas': 0}]}

In [21]:
#para guardarla en la API 
tabla_user3 = "funcion_3.csv"
tabla_user2.to_csv(tabla_user3, index=False, encoding="utf-8")

In [23]:
#guardo parquet
funcion_3= pd.read_csv("funcion_3.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/Funciones/funcion_3.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_3)
pq.write_table(table,output_file)